In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import keras
import keras.backend as k

Using TensorFlow backend.
/home/ashish/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ashish/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
import cv2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [4]:
PATH="MURA-v1.1/"

In [5]:
train_imgs_path=pd.read_csv(PATH+'train_image_paths.csv')
train_labels=pd.read_csv(PATH+'train_labeled_studies.csv')
test_imgs_path=pd.read_csv(PATH+'valid_image_paths.csv')
test_labels=pd.read_csv(PATH+'valid_labeled_studies.csv')


In [6]:
train_labels['Body_Part']=train_labels['Img_Path'].apply(lambda x: str(x.split('/')[2])[3:])
train_labels['Study_Type']=train_labels['Img_Path'].apply(lambda x: str(x.split('/')[4])[:6])
test_labels['Body_Part']=test_labels['Img_Path'].apply(lambda x: str(x.split('/')[2])[3:])
test_labels['Study_Type']=test_labels['Img_Path'].apply(lambda x: str(x.split('/')[4])[:6])

In [7]:
def read_image(Path):
    img=cv2.imread(Path)
    #img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img=cv2.resize(img,(224,224))
    #print (img.shape)
    img=np.array(img)
    #img=np.resize(img,(224,224))
    #print (img.shape)
    img=img/255.
    return img

In [8]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16,preprocess_input,decode_predictions

In [9]:
IMG_SIZE=(224,224)

In [10]:
train_datagen=ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=30,
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

In [11]:
test_datagen = ImageDataGenerator(rescale=1./255)


In [12]:
batch_size=32
import os

In [32]:
train_imgs=list(train_imgs_path.Img_Path.values)

In [34]:
valid_imgs=list(test_imgs_path.Img_Path.values)

In [13]:
train_generator = train_datagen.flow_from_directory(
        os.path.join(PATH,'train/'),  # this is the target directory
        target_size=IMG_SIZE,  # all images will be resized to 224*224
        batch_size=batch_size,
        class_mode='binary') 

Found 36808 images belonging to 7 classes.


In [14]:
validation_generator = test_datagen.flow_from_directory(
        os.path.join(PATH,'valid/'),
        target_size=IMG_SIZE,
        batch_size=batch_size,
        class_mode='binary')

Found 3197 images belonging to 7 classes.


In [15]:
model=VGG16(input_shape=(224,224,3),weights='imagenet',include_top=False)

In [16]:
model.output_shape

(None, 7, 7, 512)

In [17]:
from keras.layers import  Convolution2D,MaxPooling2D,Dense,Flatten,Dropout
from keras.models import  Sequential
from keras.optimizers import Adam

In [18]:
model1=Sequential()
model1.add(model)
model1.add(Flatten(input_shape=model.output_shape[1:]))
model1.add(Dropout(0.5))
#model1.add(Dense(128, activation='relu'))
#model1.add(Dropout(0.5))
model1.add(Dense(1, activation='softmax'))

In [19]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25089     
Total params: 14,739,777
Trainable params: 14,739,777
Non-trainable params: 0
_________________________________________________________________


In [20]:
for layer in model.layers:
    layer.trainable=False

In [21]:
model1.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-3),
              metrics=['accuracy'])

In [ ]:
model1.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)

Epoch 1/10
 2/62 [..............................] - ETA: 35:43 - loss: -33.6285 - acc: 0.2188

In [29]:
train_generator.

'MURA-v1.1/train/'